<a href="https://colab.research.google.com/github/amitkp57/Jupyter/blob/dev/Jupyter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Move to GPU mode if you are in Google Colab
Go to `Runtime` -> `Change runtime type` to activate GPU.

### Provide your credentials to the runtime


In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


### Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
DATA_DIR = "/content/gdrive/MyDrive/Data/Jupyter/data"
%cd /content/gdrive/MyDrive/Data/Jupyter/
!ls -latr

### Clone git repo

In [42]:
%cd /content/gdrive/MyDrive/Data/Jupyter/
# !git clone https://github.com/amitkp57/Jupyter
!git pull
# !pip install -r requirements.txt

/content/gdrive/MyDrive/Data/Jupyter
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0
Unpacking objects: 100% (4/4), done.
From https://github.com/amitkp57/Jupyter
   f95c2f3..f4a13c1  main       -> origin/main
Updating f95c2f3..f4a13c1
Fast-forward
 scripts/RESTUtil.py | 3 +--
 1 file changed, 1 insertion(+), 2 deletions(-)


### Setup meta data

Creates datasets.txt, tables.txt and columns.json in the /data folder. These files are used for querying Google Bigquery tables. 

In [43]:
from scripts.MetaData import save_locally
DATA_PATH='/content/gdrive/MyDrive/Data/Jupyter/data'
save_locally('/content/gdrive/MyDrive/Data/Jupyter/data')

DefaultCredentialsError: ignored